In [1]:
using Pkg, Revise
Pkg.activate("../GenLinAlgProblems/")
using GenLinAlgProblems, LinearAlgebra, Latexify, LaTeXStrings;

  Activating project at `C:\Users\jeff\NOTEBOOKS\elementary-linear-algebra\GenLinAlgProblems`


<div style="float:center;width:100%;text-align: center;"><strong style="height:60px;color:darkred;font-size:40px;">Linear Algebra Application: Cryptography</strong></div>

# 1. Hill's Cipher

## 1.1 Overview

The **Hill cipher** is an encryption method invented by Lester S. Hill in 1929.

While innovative for its time, the Hill cipher is not considered secure by modern cryptographic standards. However, it remains an important step in the history of encryption and is still studied for educational purposes

> It works by converting letters in a message to numbers and then using matrix multiplication to transform those numbers into a coded form using an invertible matrix:
> * choose a secret encoding matrix $A \in \mathbb{Z}^{N\times N}$ that has an inverse in $\mathbb{Z}^{N\times N}$
> * Let $M \in \mathbb{Z}^{K \times N}$ be the unencoded matrix representation of a given message
> * Compute the encoded message $\tilde{M} = M A$
> * The receiver of the encoded message can decoded it by computing $M = \tilde{M} A^{-1}$ and converting the resulting integer matrix back to the corresponding symbols.

## 1.2 Transform a Message into a Matrix

Consider a message, e.g., "The codeword is SECRECY".

We can transform it into a vector by associating different integers to each letter in out alphabet.<br>
E.g., we can use the unicode representation of each letter:

In [2]:
letters        = ['A', 'B', 'C', 'D', 'E']
unicode_values = [Int(letter) for letter in letters]
println( "Integer Representation of the letters $(letters)")
for i in 1:length(letters) println( "$(letters[i]): $(unicode_values[i])") end

Integer Representation of the letters ['A', 'B', 'C', 'D', 'E']
A: 65
B: 66
C: 67
D: 68
E: 69


We can use this representation to transform a given message to a vector:

In [3]:
message = "The codeword is SECRECY"
int_message = [Int(letter) for letter in message]
println("Vector representation of the message $(message)")
println(int_message)

Vector representation of the message The codeword is SECRECY
[84, 104, 101, 32, 99, 111, 100, 101, 119, 111, 114, 100, 32, 105, 115, 32, 83, 69, 67, 82, 69, 67, 89]


Finally, we choose an integer $N$ and partition the vector into sequences of length $N$,<br>
padding with 0 as required.

The result is presented as a matrix $M$.

In [4]:
N      = 5

N_rows = cld(length(int_message), N)     # Calculate the number of rows required
desired_length     = N_rows * N          # Pad the message vector with zeros if necessary
padded_int_message = [int_message; zeros(Int, desired_length - length(int_message))]

M = reshape(padded_int_message, N, N_rows)' # Reshape into a matrix
latexify([latex(raw"\text{Matrix representation of the message: } M ="), M']')

L"\begin{equation}
\left[
\begin{array}{cc}
\text{Matrix representation of the message: } M = & \left[
\begin{array}{ccccc}
84 & 104 & 101 & 32 & 99 \\
111 & 100 & 101 & 119 & 111 \\
114 & 100 & 32 & 105 & 115 \\
32 & 83 & 69 & 67 & 82 \\
69 & 67 & 89 & 0 & 0 \\
\end{array}
\right] \\
\end{array}
\right]
\end{equation}
"

## 1.3 Encode the Message

To encode the message, we choose an invertible matrix $A$ of size $N \times N$,<br>
and multiply each row of the message matrix $M$ by $A$,<br>
i.e., we compute $\tilde{M} = M A$

In [5]:
A, A⁻¹ = gen_inv_pb( N )

latexify( [latex(raw"\text{Secret Encoding Matrix } A ="), A']')

L"\begin{equation}
\left[
\begin{array}{cc}
\text{Secret Encoding Matrix } A = & \left[
\begin{array}{ccccc}
1 & -1 & -2 & 0 & -2 \\
-1 & 2 & 0 & -1 & 0 \\
-3 & 4 & 5 & -1 & 6 \\
2 & -1 & -4 & 0 & -4 \\
1 & 1 & -6 & -2 & -5 \\
\end{array}
\right] \\
\end{array}
\right]
\end{equation}
"

In [6]:
M̃ = M * A
latexify( [latex( raw"\text{Encoded Message } \tilde{M} = "), M̃']')

L"\begin{equation}
\left[
\begin{array}{cc}
\text{Encoded Message } \tilde{M} =  & \left[
\begin{array}{ccccc}
-160 & 595 & -385 & -403 & -185 \\
57 & 485 & -859 & -423 & -647 \\
243 & 224 & -1178 & -362 & -1031 \\
-42 & 425 & -479 & -316 & -328 \\
-265 & 421 & 307 & -156 & 396 \\
\end{array}
\right] \\
\end{array}
\right]
\end{equation}
"

Note that the entries in the encoded message do not correspond to letters directly.

## 1.4 Decode the Message

The received message $\tilde{M}$ is easy to decode given knowledge of the encoding key $A$:<br>
$\qquad M = \tilde{M} A^{-1},$<br> which can then be translated back to text<br>
by lookup of the symbols corresponding to their integer representation:

In [7]:
function decipher( M̃, A⁻¹ )
    M = M̃ * A⁻¹
    join(Char.(vec(M')))
end
decipher( M̃, A⁻¹ )

"The codeword is SECRECY\0\0"

# 2. Avoiding the Computation of a Matrix Inverse

The inverse of the encoding matrix $A$ can of course be computed exactly,<br>
but one would normally avoid computing it unless absolutely necessary.

Instead, we can use the LU decomposition and use forward/backsubstitution for all messages.

Since $M = \tilde{M} A^{-1} \Leftrightarrow A^t M^t= \tilde{M}^t,\quad$ we can solve for $M$ using the LU decomposition of $A^t$.

Note the use of the transpose to write the problem in standard form (that way, the unknown matrix is to the right)

In [8]:
LU_Aᵗ = lu(Rational{Int}.(A'))  # compute the LU decomposition of A'

function lu_decipher( M̃, LU_Aᵗ )
    Mᵗ = LU_Aᵗ \ M̃'
    join(Char.(vec(Mᵗ)))
end
lu_decipher( M̃, LU_Aᵗ )

"The codeword is SECRECY\0\0"